# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

03-03-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-03-04 04:21:09,33.93911,67.709953,174214,7619,NaN,NaN,Afghanistan,447.524851,4.373357
1,NaN,NaN,NaN,Albania,2022-03-04 04:21:09,41.15330,20.168300,271825,3474,NaN,NaN,Albania,9445.583432,1.278028
2,NaN,NaN,NaN,Algeria,2022-03-04 04:21:09,28.03390,1.659600,265130,6848,NaN,NaN,Algeria,604.615037,2.582884
3,NaN,NaN,NaN,Andorra,2022-03-04 04:21:09,42.50630,1.521800,38342,151,NaN,NaN,Andorra,49624.021226,0.393824
4,NaN,NaN,NaN,Angola,2022-03-04 04:21:09,-11.20270,17.873900,98746,1900,NaN,NaN,Angola,300.447863,1.924129


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,172716,172901,173047,173084,173146,173395,173659,173879,174073,174214
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,270455,270734,270947,271141,271141,271527,271563,271702,271825,271825
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,264365,264488,264603,264706,264778,264855,264936,265010,265079,265130


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7569,7574,7578,7579,7585,7593,7598,7609,7617,7619
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3451,3453,3457,3458,3458,3464,3469,3472,3474,3474
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6812,6816,6820,6823,6828,6831,6835,6840,6843,6848


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15442,15451,15468,15479,15503,15509,15510,15520,15528,15526
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,54805,54837,54874,54904,54957,54972,54978,54987,55040,55061
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5430,5433,5436,5438,5440,5445,5445,5445,5446,5447


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,184,184,192,194,194,194,194,195,198,199
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,635,636,640,640,640,640,640,641,644,645
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,92,92,92,93,93,93,93,93,93,94


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
674,1001.0,Autauga,Alabama,US,2022-03-04 04:21:09,32.539527,-86.644082,15526,199,NaN,NaN,"Autauga, Alabama, US",27790.008771,1.281721
711,1075.0,Lamar,Alabama,US,2022-03-04 04:21:09,33.779950,-88.096680,3663,63,NaN,NaN,"Lamar, Alabama, US",26533.864542,1.719902
712,1077.0,Lauderdale,Alabama,US,2022-03-04 04:21:09,34.901719,-87.656247,23941,370,NaN,NaN,"Lauderdale, Alabama, US",25818.244562,1.545466


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,79196008,956261,0.0
India,42951556,514589,0.0
Brazil,28906672,650824,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,79196008,956261,0.0,78239747.0,2022-03-04 04:21:09,37.934269,-91.444083
India,42951556,514589,0.0,42436967.0,2022-03-04 04:21:09,23.088275,81.806127
Brazil,28906672,650824,0.0,28255848.0,2022-03-04 04:21:09,-12.669522,-48.480493
France,23079453,139962,0.0,22939491.0,2022-03-04 04:21:09,6.430808,-34.730285
United Kingdom,19211944,162472,0.0,19049472.0,2022-03-04 04:21:09,30.654806,-30.248397


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,8997446,86026,0.0
Texas,6630636,85378,0.0
Florida,5844096,70789,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,8997446,86026,0.0,8911420.0,2022-03-04 04:21:09,37.843962,-120.728594
Texas,6630636,85378,0.0,6545258.0,2022-03-04 04:21:09,31.660643,-98.653069
Florida,5844096,70789,0.0,5773307.0,2022-03-04 04:21:09,28.940755,-82.700744
New York,4930987,67479,0.0,4863508.0,2022-03-04 04:21:09,42.544151,-75.474183
Illinois,3035870,35319,0.0,3000551.0,2022-03-04 04:21:09,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,2800741,30911,0.0
Arizona,Maricopa,1248818,15625,0.0
Florida,Miami-Dade,1176706,10320,0.0
Illinois,Cook,1113031,14099,0.0
Texas,Harris,993328,10693,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,2800741,30911,0.0,2769830.0,2022-03-04 04:21:09,34.308284,-118.228241,6037.0
Arizona,Maricopa,1248818,15625,0.0,1233193.0,2022-03-04 04:21:09,33.348359,-112.491815,4013.0
Florida,Miami-Dade,1176706,10320,0.0,1166386.0,2022-03-04 04:21:09,25.611236,-80.551706,12086.0
Illinois,Cook,1113031,14099,0.0,1098932.0,2022-03-04 04:21:09,41.841448,-87.816588,17031.0
Texas,Harris,993328,10693,0.0,982635.0,2022-03-04 04:21:09,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,173879,271702,265010,38165,98746,11,7447,8904176,420156,3256772,...,844400,236501,19,515124,3557629,647203,524,11771,312970,236871
2022-03-02,174073,271825,265079,38249,98746,11,7449,8912317,420498,3296784,...,846868,236596,19,515582,3709481,648039,524,11771,313203,237503
2022-03-03,174214,271825,265130,38342,98746,11,7449,8921536,420757,3321943,...,849050,236687,19,515943,3885631,649110,526,11772,313394,237503


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,7609,3472,6840,151,1900,0,135,126257,8478,5269,...,6991,1636,1,5636,40338,5513,0,2135,3953,5395
2022-03-02,7617,3474,6843,151,1900,0,135,126390,8493,5316,...,7005,1637,1,5639,40452,5532,0,2135,3955,5396
2022-03-03,7619,3474,6848,151,1900,0,135,126531,8505,5354,...,7014,1637,1,5643,40547,5542,0,2135,3957,5396


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15442,15451,15468,15479,15503,15509,15510,15520,15528,15526
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,3592,3598,3613,3635,3644,3644,3644,3657,3660,3663
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,23793,23828,23861,23884,23899,23908,23910,23923,23934,23941
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,7506,7515,7527,7536,7546,7561,7562,7566,7576,7583
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,37454,37472,37484,37497,37508,37512,37520,37525,37533,37541


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-03-01,1280351,239494,84,1976890,820677,8981736,1314678,723559,256656,49,...,2006591,6622301,923130,112364,15402,1641438,1423990,490544,1572682,155184
2022-03-02,1281610,240085,146,1976890,821236,8987156,1315727,723812,256753,49,...,2008395,6626391,923591,112364,15410,1643050,1427013,491042,1573463,155277
2022-03-03,1282372,240085,155,1980769,821717,8997446,1316450,724401,256895,49,...,2010068,6630636,923987,112906,15412,1644694,1427013,491698,1574039,155349


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-03-01       15520   54987    5445  6375  14772    2279   5037   31753   
2022-03-02       15528   55040    5446  6375  14786    2280   5037   31802   
2022-03-03       15526   55061    5447  6378  14799    2280   5037   31871   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-03-01         8433     5086  ...    6707   1892     8025     1917   
2022-03-02         8434     5088  ...    6715   1892     8022     1917   
2022-03-03         8435     5090  ...    6718   1892     8025     1918   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-03-01          11003  9791  5627          0     2340   1548  
2022-03-02          11011  9794  5627          0     2343   1562  
2022-03-03          11014  9805  5624          0     2343   1562  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-03-01,18287,1163,0,27946,10579,85557,11794,10471,2711,0,...,24495,85050,4417,598,109,18811,11866,6339,13351,1741
2022-03-02,18335,1191,0,27946,10611,85827,11801,10512,2711,0,...,24550,85260,4436,598,109,18859,11954,6381,13398,1741
2022-03-03,18407,1191,0,27708,10661,86026,11809,10515,2711,0,...,24638,85378,4442,604,109,18905,11954,6427,13450,1741


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-03-01         195     641      93   99    219      49    119     588   
2022-03-02         198     644      93   99    220      49    119     589   
2022-03-03         199     645      94   99    221      49    120     589   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-03-01          153       78  ...     136     39       74       27   
2022-03-02          153       79  ...     136     39       74       27   
2022-03-03          153       79  ...     136     39       74       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-03-01            123    16    37          0       43     18  
2022-03-02            123    16    37          0       43     18  
2022-03-03            123    16    37          0       43     18  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,0.001267,0.000512,0.000279,0.004369,0.000051,0.0,0.001345,0.000395,0.000772,0.006725,...,0.002300,0.000406,0.0,0.000000,0.033148,0.001944,0.005758,0.000000,0.000703,0.002077
2022-03-02,0.001116,0.000453,0.000260,0.002201,0.000000,0.0,0.000269,0.000914,0.000814,0.012286,...,0.002923,0.000402,0.0,0.000889,0.042683,0.001292,0.000000,0.000000,0.000744,0.002668
2022-03-03,0.000810,0.000000,0.000192,0.002431,0.000000,0.0,0.000000,0.001034,0.000616,0.007631,...,0.002577,0.000385,0.0,0.000700,0.047486,0.001653,0.003817,0.000085,0.000610,0.000000


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,0.001448,0.000865,0.000732,0.0,0.0,NaN,0.0,0.000832,0.002009,0.011324,...,0.000859,0.000612,0.0,0.000000,-0.004541,0.002181,NaN,0.0,0.000253,0.000000
2022-03-02,0.001051,0.000576,0.000439,0.0,0.0,NaN,0.0,0.001053,0.001769,0.008920,...,0.002003,0.000611,0.0,0.000532,0.002826,0.003446,NaN,0.0,0.000506,0.000185
2022-03-03,0.000263,0.000000,0.000731,0.0,0.0,NaN,0.0,0.001116,0.001413,0.007148,...,0.001285,0.000000,0.0,0.000709,0.002348,0.001808,NaN,0.0,0.000506,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-03-01,0.000614,0.000000,0.000000,0.000000,0.000845,0.000442,0.000684,0.000235,0.000136,0.0,...,0.001202,0.000785,0.000301,0.000000,0.000260,0.000912,0.000000,0.000883,0.000436,0.000535
2022-03-02,0.000983,0.002468,0.738095,0.000000,0.000681,0.000603,0.000798,0.000350,0.000378,0.0,...,0.000899,0.000618,0.000499,0.000000,0.000519,0.000982,0.002123,0.001015,0.000497,0.000599
2022-03-03,0.000595,0.000000,0.061644,0.001962,0.000586,0.001145,0.000550,0.000814,0.000553,0.0,...,0.000833,0.000641,0.000429,0.004824,0.000130,0.001001,0.000000,0.001336,0.000366,0.000464


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-03-01      0.000645  0.000164  0.000000  0.000000  0.000813  0.000000   
2022-03-02      0.000515  0.000964  0.000184  0.000000  0.000948  0.000439   
2022-03-03     -0.000129  0.000382  0.000184  0.000471  0.000879  0.000000   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2022-03-01      0.000397  0.003635  0.000237  0.002365  ...  0.000746    0.0   
2022-03-02      0.000000  0.001543  0.000119  0.000393  ...  0.001193    0.0   
2022-03-03      0.000000  0.002170  0.000119  0.000393  ...  0.000447    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2022-03-01      0.000249  0.001044   0.000364  0.000920  0.000711        NaN   
2022-03-02     -0.000374  0.000000   0.000727  0.000306  0.000000        NaN   
2022-03-03      0.000374  0.000522   0.000272  0.001123 -0.000533        NaN   

Province_State                      
Admin2          Washakie    Weston  
2022-03-01      0.001284  0.000000  
2022-03-02      0.001282  0.009044  
2022-03-03      0.000000  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-03-01,0.003402,0.000000,NaN,0.000000,0.005226,0.001041,0.001869,0.000096,0.000738,NaN,...,0.001063,0.001779,0.001360,0.000000,0.0,0.002131,0.000000,0.003483,0.004061,0.00288
2022-03-02,0.002625,0.024076,NaN,0.000000,0.003025,0.003156,0.000594,0.003916,0.000000,NaN,...,0.002245,0.002469,0.004302,0.000000,0.0,0.002552,0.007416,0.006626,0.003520,0.00000
2022-03-03,0.003927,0.000000,NaN,-0.008516,0.004712,0.002319,0.000678,0.000285,0.000000,NaN,...,0.003585,0.001384,0.001353,0.010033,0.0,0.002439,0.000000,0.007209,0.003881,0.00000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin   Barbour Bibb    Blount Bullock    Butler   
2022-03-01      0.005155  0.001562  0.000000  0.0  0.004587     0.0  0.000000   
2022-03-02      0.015385  0.004680  0.000000  0.0  0.004566     0.0  0.000000   
2022-03-03      0.005051  0.001553  0.010753  0.0  0.004545     0.0  0.008403   

Province_State                                ...   Wyoming                  \
Admin2           Calhoun  Chambers  Cherokee  ...      Park Platte Sheridan   
2022-03-01      0.000000  0.006579  0.000000  ...  0.022556    0.0      0.0   
2022-03-02      0.001701  0.000000  0.012821  ...  0.000000    0.0      0.0   
2022-03-03      0.000000  0.000000  0.000000  ...  0.000000    0.0      0.0   

Province_State                                                                 
Admin2         Sublette Sweetwater Teton     Uinta Unassigned Washakie Weston  
2022-03-01          0.0        0.0   0.0  0.027778       -1.0      0.0    0.0  
2022-03-02          0.0        0.0   0.0  0.000000        NaN      0.0    0.0  
2022-03-03          0.0        0.0   0.0  0.000000        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,0.001256,0.000515,0.000297,0.002283,0.000131,0.0,0.000717,0.000460,0.000746,0.006983,...,0.002562,0.000458,0.005850,0.000563,0.033267,0.003322,0.005563,0.000171,0.000522,0.001862
2022-03-02,0.001186,0.000484,0.000279,0.002242,0.000065,0.0,0.000493,0.000687,0.000780,0.009634,...,0.002743,0.000430,0.002925,0.000726,0.037975,0.002307,0.002782,0.000086,0.000633,0.002265
2022-03-03,0.000998,0.000242,0.000236,0.002337,0.000033,0.0,0.000246,0.000861,0.000698,0.008633,...,0.002660,0.000407,0.001463,0.000713,0.042731,0.001980,0.003299,0.000085,0.000622,0.001133


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,0.001102,0.001055,0.000649,0.000026,0.000132,NaN,4.792048e-07,0.000649,0.001888,0.008657,...,0.001247,0.000991,0.0,0.000295,0.000654,0.003168,NaN,0.000276,0.000226,0.000124
2022-03-02,0.001077,0.000815,0.000544,0.000013,0.000066,NaN,2.396024e-07,0.000851,0.001829,0.008788,...,0.001625,0.000801,0.0,0.000414,0.001740,0.003307,NaN,0.000138,0.000366,0.000154
2022-03-03,0.000670,0.000408,0.000637,0.000007,0.000033,NaN,1.198012e-07,0.000983,0.001621,0.007968,...,0.001455,0.000401,0.0,0.000562,0.002044,0.002558,NaN,0.000069,0.000436,0.000077


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-03-01,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-03-02,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-03-03,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-03-01,0.000496,0.001033,0.006977,0.000108,0.000771,0.000649,0.000928,0.000508,0.000287,1.975037e-216,...,0.000736,0.000666,0.000433,0.000897,0.000381,0.001056,0.001243,0.001467,0.000498,0.000632
2022-03-02,0.000740,0.001750,0.372536,0.000054,0.000726,0.000626,0.000863,0.000429,0.000332,9.875185e-217,...,0.000817,0.000642,0.000466,0.000448,0.000450,0.001019,0.001683,0.001241,0.000497,0.000616
2022-03-03,0.000667,0.000875,0.217090,0.001008,0.000656,0.000886,0.000706,0.000621,0.000443,4.937592e-217,...,0.000825,0.000641,0.000448,0.002636,0.000290,0.001010,0.000842,0.001288,0.000432,0.000540


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-03-01      0.000532  0.000239  0.000164  0.000095  0.000641  0.000027   
2022-03-02      0.000524  0.000602  0.000174  0.000048  0.000795  0.000233   
2022-03-03      0.000197  0.000492  0.000179  0.000259  0.000837  0.000116   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-03-01      0.000249  0.002059  0.000227  0.001289  ...  0.000641   
2022-03-02      0.000125  0.001801  0.000173  0.000841  ...  0.000917   
2022-03-03      0.000062  0.001985  0.000146  0.000617  ...  0.000682   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-03-01      0.000574  0.000474  0.000782   0.000411  0.000917  0.000705   
2022-03-02      0.000287  0.000050  0.000391   0.000569  0.000612  0.000353   
2022-03-03      0.000144  0.000212  0.000456   0.000421  0.000867 -0.000090   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-03-01           -1.0  0.000761  0.001197  
2022-03-02           -1.0  0.001021  0.005120  
2022-03-03           -1.0  0.000511  0.002560  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-03-01,0.002097,0.000152,NaN,0.000179,0.004106,0.001986,0.001416,0.000293,0.000493,NaN,...,0.001201,0.001406,0.000995,0.001475,0.000038,0.004565,0.001103,0.002363,0.003000,0.004127
2022-03-02,0.002361,0.012114,NaN,0.000089,0.003566,0.002571,0.001005,0.002104,0.000247,NaN,...,0.001723,0.001938,0.002648,0.000738,0.000019,0.003558,0.004259,0.004494,0.003260,0.002063
2022-03-03,0.003144,0.006057,NaN,-0.004214,0.004139,0.002445,0.000841,0.001195,0.000123,NaN,...,0.002654,0.001661,0.002000,0.005386,0.000009,0.002999,0.002130,0.005852,0.003571,0.001032


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2022-03-01      0.003589  0.000894  0.000343  0.000005  0.002729   
2022-03-02      0.009487  0.002787  0.000172  0.000003  0.003648   
2022-03-03      0.007269  0.002170  0.005462  0.000001  0.004097   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2022-03-01      3.922331e-10  0.001210  0.000365  0.003195  0.000408  ...   
2022-03-02      1.961166e-10  0.000605  0.001033  0.001598  0.006614  ...   
2022-03-03      9.805828e-11  0.004504  0.000516  0.000799  0.003307  ...   

Province_State   Wyoming                                                   \
Admin2              Park        Platte  Sheridan      Sublette Sweetwater   
2022-03-01      0.011338  6.274566e-09  0.000110  1.183290e-06   0.000033   
2022-03-02      0.005669  3.137283e-09  0.000055  5.916452e-07   0.000016   
2022-03-03      0.002835  1.568642e-09  0.000028  2.958226e-07   0.000008   

Province_State                                                     
Admin2             Teton     Uinta Unassigned  Washakie    Weston  
2022-03-01      0.000260  0.013889       -1.0  0.000093  0.000230  
2022-03-02      0.000130  0.006944       -1.0  0.000047  0.000115  
2022-03-03      0.000065  0.003472       -1.0  0.000023  0.000057  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 